# App wrapper

In [1]:
import time
import osmnx as ox
import pandas as pd


import sys
sys.path.append('../../Multimodal_freight_USA/')
from mfreight.Multimodal.graph_utils import MultimodalNet
from mfreight.utils import build_graph

departure_x, departure_y = (30.439440, -85.057166)
arrival_x, arrival_y = (25.382380, -80.475159)

In [2]:
departure = (departure_x, departure_y)
arrival = (arrival_x, arrival_y)

In [3]:
Net = MultimodalNet(path_u= "../../Multimodal_freight_USA/mfreight/multimodal/data/multimodal_G_tot_u_w_price.plk")

In [4]:
def app_result(departure, arrival, Net, feature, operators= ["BNSF", "UP", "CN", "CPRS", "KCS", "CSXT", "NS"]):
    price_target = Net.get_price_target(departure, arrival)

    removed_edges, removed_nodes = Net.chose_operator_in_graph(operators)

    path = Net.get_shortest_path(
        departure,
        arrival,
        target_weight=feature,
        price_target=price_target
    )

    scanned_route, path_rail_edges, stop_list = Net.scan_route(
        Net.route_detail_from_graph(
            path, show_entire_route=True, price_target=price_target
        )
    )

    route_details = Net.route_detail_from_graph(path, price_target=price_target)

    chosen_mode = Net.chosen_path_mode(route_details)

    route_specs = Net.compute_all_paths(
        departure=departure,
        arrival=arrival,
        price_target=price_target,
    )


    build_graph.add_nodes_from_df(Net.G_multimodal_u, removed_nodes)
    build_graph.add_edges_from_df(Net.G_multimodal_u, removed_edges)
    return price_target, route_specs

In [5]:
price_target, route_specs = app_result(departure, arrival, Net, feature='price')

In [6]:
route_specs

,CO2_eq_kg,duration_h,price
Multimodal,16.8,44.4,1317.9
Truckload,26.3,7.5,1042.6


In [7]:
price_target

'range1'

In [8]:
# Geographical
departures =[(34.186234, -118.599367),
(33.751475, -84.468668),
(43.097041, -70.801162),
(34.935067, -89.991454),
(25.821497, -80.312094),
(41.887051, -87.693805),
(25.821497, -80.312094),
(47.457887, -122.260722),
(45.558748, -122.650805),
(39.644398, -105.006873)]

arrivals = [(33.751475, -84.468668),
(34.186234, -118.599367),
(34.935067, -89.991454),
(43.097041, -70.801162),
(41.887051, -87.693805),
(25.821497, -80.312094),
(47.457887, -122.260722),
(25.821497, -80.312094),
(39.644398, -105.006873),
(45.558748, -122.650805)]

In [16]:
route_specs_df = pd.DataFrame()
import time
for departure, arrival in zip(departures, arrivals):
    time.sleep(5)
    price_target, route_specs = app_result(departure, arrival, Net, feature='CO2_eq_kg')
                
    route_specs_df = route_specs_df.append({'duration':route_specs.loc['Multimodal', 'duration_h'],
                                            'price':route_specs.loc['Multimodal', 'price']},
                                           ignore_index=True)
    print(price_target)

('CA', 'GA')
('GA', 'CA')
range2
range2
range2
range2
('FL', 'IL')
('IL', 'FL')
range5
range5


In [14]:
route_specs_df.price


0    3637.6
1    2274.3
2    2292.9
3    2292.9
4    2458.6
5    2458.6
6    1334.0
7    2504.5
8    3921.8
9    3921.8
Name: price, dtype: float64

In [18]:
departures = [(41.351553, -82.067394),
(41.182853, -83.728211),
(25.823094, -80.306578),
(25.823094, -80.306578),
(25.823094, -80.306578)]

arrivals = [(27.938252, -81.698119),
(27.938252, -81.698120),
(40.800558, -74.024744),
(40.757823, -73.780038),
(41.057003, -74.750988)]

In [19]:
route_specs_df = pd.DataFrame()
import time
for departure, arrival in zip(departures, arrivals):
    time.sleep(5)
    price_target, route_specs = app_result(departure, arrival, Net, feature='CO2_eq_kg')
                
    route_specs_df = route_specs_df.append({'duration':route_specs.loc['Multimodal', 'duration_h'],
                                            'price':route_specs.loc['Multimodal', 'price']},
                                           ignore_index=True)
    print(price_target)

range2
range2
('FL', 'NJ')
range2
('FL', 'NJ')


In [22]:
route_specs_df.duration

0     97.9
1     93.8
2    104.7
3    107.0
4    107.1
Name: duration, dtype: float64

In [23]:
route_specs_df.price

0    2143.1
1    1930.2
2    2085.6
3    2332.2
4    2145.1
Name: price, dtype: float64

In [17]:
import geopandas as gpd

In [18]:
df = gpd.read_file('../../Multimodal_freight_USA/mfreight/Rail/rail_data/Intermodal stations/Intermodal_Freight_Facilities_RailTOFCCOFC.shp')

In [19]:
df

,OBJECTID,TERMINAL,TERM_ADDRE,CITY,STATE,ZIP_CODE,PORT,DATE,RAIL_COMPA,EQUIPMENT,ACCESS_ROA,SPLC,LAT,LON,geometry
0,1,"Albany, NY - Mechanicville",50 Route 67,Mechanicville,NY,12118,None,2018-08-01,NS,COFC,None,171760,42.914715,-73.709533,POINT (-73.70953 42.91471)
1,2,"Albuquerque, NM",100 Woodward Road S.E.,Albuquerque,NM,87102,None,2018-09-01,BNSF,TOFC/COFC,None,774900,35.047580,-106.653344,POINT (-106.65334 35.04758)
2,3,"Alliance, TX",1111 Intermodal Parkway,Haslet,TX,76052,None,2018-09-02,BNSF,TOFC/COFC,None,668100,32.999881,-97.347026,POINT (-97.34703 32.99988)
3,4,"Arcadia, WI",One Ashley Way,Arcadia,WI,54612,None,2018-09-01,CN,TOFC/COFC,South Washington St.,328754,44.251243,-91.509663,POINT (-91.50966 44.25124)
4,5,"Atlanta, GA - Austell","6000 Dr. Luke Glenn Garrett, Jr. Memorial Highway",Austell,GA,30106,None,2018-08-01,NS,TOFC/COFC,None,456496,33.825798,-84.655069,POINT (-84.65507 33.82580)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,177,"Wilmington, CA - TraPac Intermodal Container T...",630 West Harry Bridges Boulevard,Wilmington,CA,90744,Port of Los Angeles,2018-12-07,PHL,TOFC/COFC,None,883257,33.770320,-118.273289,POINT (-118.27329 33.77032)
177,178,"Wilmington, NC - Container Terminal",1 Shipyard Boulevard,Wilmington,NC,28401,Port of Wilmington,2018-09-11,CSX,COFC,None,409160,34.191224,-77.952419,POINT (-77.95242 34.19122)
178,179,"Winter Haven, FL - Central Florida Intermodal ...",3935 Intermodal Drive,Winter Haven,FL,33884,None,2018-09-11,"CSX, EVWR",COFC,None,497540,27.939953,-81.693226,POINT (-81.69323 27.93995)
179,180,"Worcester, MA",165 Grafton Street,Worcester,MA,01604,None,2018-09-11,CSX,TOFC/COFC,None,145460,42.259951,-71.789244,POINT (-71.78924 42.25995)
